
https://ziglang.org/learn/samples/

<https://blog.logrocket.com/getting-started-zig-programming-language/>
`snap install zig --classic --beta`

`zig init-exe`
`zig translate-c`

```zig
//re
const std = @import("std");

pub fn main() void {
    std.debug.print("Hello Zig!\n", .{});
    std.debug.print("Hello {}\n", .{2023}); // Hello 2023
    std.debug.print("Hello {s}! {d}\n", .{"Zig", 2023}); // Hello Zig! 2023

    var x: i8 = -100;     // Signed 8-bit integer
    var y: u8 = 120;      // Unsigned 8-bit integer
    var z: f32 = 100.324; // 32-bit floating point

    std.debug.print("x={}\n", .{x});        // x=-100
    std.debug.print("y={}\n", .{y});        // y=120
    std.debug.print("z={d:3.2}\n", .{z});   // z=100.32
    
    const LogType = enum {
        info,
        err,
        warn
    };

    const ltInfo = LogType.info;
    const ltErr = LogType.err;

    std.debug.print("{}\n", .{ltInfo}); // main.main.LogType.info
    std.debug.print("{}\n", .{ltErr});  // main.main.LogType.err

}
fn add(a: i8, b: i8) i8 {
    return a + b;
}
```

zigzap/zap: A micro-framework for building web backends
JakubSzark/zig-string: A string library for Zig
kooparse/zalgebra: Linear algebra library for games and real-time graphics
zigimg/zigimg: Zig library for reading and writing different image formats
ziglibs/ini: A simple INI parser for Zig

<https://github.com/C-BJ/awesome-zig>

Good C interop supposedly. Such that you don't really need to write bindings?
`c_int`

No libc by default but easy to add in

```zig

```

ReleaseSafe mode detects undefined behavior.

# Comptime

Cool oppurtunirty for some staged metapgoramming stuff right?
Like KMP, parsers, etc
Generics are calculated types which feels a bit mindblowing (?)
Maybe that is'nt that crazy from system f perspective

# Allocators


In [8]:
! zig help

Usage: zig [command] [options]

Commands:

  build            Build project from build.zig
  fetch            Copy a package into global cache and print its hash
  init             Initialize a Zig package in the current directory

  build-exe        Create executable from source or object files
  build-lib        Create library from source or object files
  build-obj        Create object from source or object files
  test             Perform unit testing
  run              Create executable and run immediately

  ast-check        Look for simple compile errors in any set of files
  fmt              Reformat Zig source into canonical form
  reduce           Minimize a bug report
  translate-c      Convert C code to Zig code

  ar               Use Zig as a drop-in archiver
  cc               Use Zig as a drop-in C compiler
  c++              Use Zig as a drop-in C++ compiler
  dlltool          Use Zig as a drop-in dlltool.exe
  lib              Use Zig as a drop-in lib.exe
  ranlib    

In [2]:
%%file /tmp/hello.zig

const std = @import("std");

pub fn main() !void {
    std.debug.print("Hello World!\n", .{});
}

Overwriting /tmp/hello.zig


In [4]:
! zig run /tmp/hello.zig 

Hello World!


In [8]:
%%file /tmp/hello2.zig
const std = @import("std");

// longer form version. Actually zig has some stderr and stout distinction
pub fn main() !void {
    try std.fs.File.stdout().writeAll("hello world!\n");
}

Overwriting /tmp/hello2.zig


In [20]:
! zig run /tmp/hello2.zig

hello world!


In [21]:
%%file /tmp/overflow.zig
test "integer overflow at compile time" {
    const x: u8 = 255;
    _ = x + 1;
}

Writing /tmp/overflow.zig


In [22]:
! zip test /tmp/overflow.zig

  adding: tmp/overflow.zig (deflated 6%)


In [23]:
%%file /tmp/memoryleak.zig
const std = @import("std");

pub fn main() !void {
    var general_purpose_allocator = std.heap.GeneralPurposeAllocator(.{}){};
    defer std.debug.assert(general_purpose_allocator.deinit() == .ok);

    const gpa = general_purpose_allocator.allocator();

    const u32_ptr = try gpa.create(u32);
    _ = u32_ptr; // silences unused variable error

    // oops I forgot to free!
}

Writing /tmp/memoryleak.zig


In [25]:
! zig run /tmp/memoryleak.zig

error(gpa): memory address 0x70f8583c0000 leaked: 
/tmp/memoryleak.zig:9:35: 0x113d457 in main (memoryleak.zig)
    const u32_ptr = try gpa.create(u32);
                                  ^
/snap/zig/14958/lib/std/start.zig:627:37: 0x113dd49 in posixCallMainAndExit (std.zig)
            const result = root.main() catch |err| {
                                    ^
/snap/zig/14958/lib/std/start.zig:232:5: 0x113d331 in _start (std.zig)
    asm volatile (switch (native_arch) {
    ^

thread 146160 panic: reached unreachable code
/snap/zig/14958/lib/std/debug.zig:559:14: 0x1044179 in assert (std.zig)
    if (!ok) unreachable; // assertion failure
             ^
/tmp/memoryleak.zig:5:27: 0x113d526 in main (memoryleak.zig)
    defer std.debug.assert(general_purpose_allocator.deinit() == .ok);
                          ^
/snap/zig/14958/lib/std/start.zig:627:37: 0x113dd49 in posixCallMainAndExit (std.zig)
            const result = root.main() catch |err| {
                                    

In [ ]:
%%file /tmp/raylib.zig
const ray = @cImport({
    @cInclude("raylib.h");
});

pub fn main() void {
    const screenWidth = 800;
    const screenHeight = 450;

    ray.InitWindow(screenWidth, screenHeight, "raylib [core] example - basic window");
    defer ray.CloseWindow();

    ray.SetTargetFPS(60);

    while (!ray.WindowShouldClose()) {
        ray.BeginDrawing();
        defer ray.EndDrawing();

        ray.ClearBackground(ray.RAYWHITE);
        ray.DrawText("Hello, World!", 190, 200, 20, ray.LIGHTGRAY);
    }
}

Writing /tmp/raylib.zig


In [29]:
! zig run /tmp/raylib.zig -lc -lraylib

INFO: Initializing raylib 5.6-dev
INFO: Platform backend: DESKTOP (GLFW)
INFO: Supported raylib modules:
INFO:     > rcore:..... loaded (mandatory)
INFO:     > rlgl:...... loaded (mandatory)
INFO:     > rshapes:... loaded (optional)
INFO:     > rtextures:. loaded (optional)
INFO:     > rtext:..... loaded (optional)
INFO:     > rmodels:... loaded (optional)
INFO:     > raudio:.... loaded (optional)
INFO: DISPLAY: Device initialized successfully
INFO:     > Display size: 1920 x 1080
INFO:     > Screen size:  800 x 450
INFO:     > Render size:  800 x 450
INFO:     > Viewport offsets: 0, 0
INFO: GLAD: OpenGL extensions loaded successfully
INFO: GL: Supported extensions count: 238
INFO: GL: OpenGL device information:
INFO:     > Vendor:   AMD
INFO:     > Renderer: AMD Radeon 780M (radeonsi, gfx1103_r1, LLVM 19.1.1, DRM 3.61, 6.14.0-27-generic)
INFO:     > Version:  4.6 (Core Profile) Mesa 24.2.8-1ubuntu1~24.04.1
INFO:     > GLSL:     4.60
INFO: GL: VAO extension detected, VAO functions load

In [17]:
%%file /tmp/z3.zig

const std = @import("std");
const z3 = @cImport({
    @cInclude("z3.h");
});

pub fn main() !void {
    const ctx = z3.Z3_mk_context(null);
    const solver = z3.Z3_mk_solver(ctx);
    const xsym = z3.Z3_mk_string_symbol(ctx, "x");
    const ysym = z3.Z3_mk_string_symbol(ctx, "y");
    const intsort = z3.Z3_mk_int_sort(ctx);
    const x = z3.Z3_mk_const(ctx, xsym, intsort);
    const y = z3.Z3_mk_const(ctx, ysym, intsort);
    const constraint = z3.Z3_mk_eq(ctx, x, y);
    z3.Z3_solver_assert(ctx, solver, constraint);
    const result = z3.Z3_solver_check(ctx, solver);
    if (result == z3.Z3_L_TRUE) {
        std.debug.print("Satisfiable\n", .{});
    } else {
        std.debug.print("Unsatisfiable\n", .{});
    }
}


Overwriting /tmp/z3.zig


In [19]:
! zig run /tmp/z3.zig -lc -lz3

Satisfiable


In [30]:
%%file /tmp/leantest.lean

@[export myadd]
def myadd (x y : UInt64) : UInt64 := x + y

Writing /tmp/leantest.lean


In [31]:
! lean -c /tmp/myadd.c /tmp/leantest.lean

In [44]:
%%file /tmp/lean.zig
const std = @import("std");
const lean = @cImport({
    @cInclude("lean/lean.h");
});
//const leantest = @cImport({
//    @cInclude("/tmp/myadd.c");
//});

pub fn main() !void {
    lean.lean_initialize();
    defer lean.lean_finalize();
    //const x: u64 = 3;
    //const y: u64 = 4;
    //const result = leantest.myadd(x, y);
    //std.debug.print("myadd({},{}) = {}\n", .{x, y, result});
}   

Overwriting /tmp/lean.zig


In [45]:
! zig run /tmp/lean.zig -lc -I$(lean --print-prefix)/include -L$(lean --print-libdir) -lleanshared -lleanshared_1

[49/1200] Linking
[1505] Semantic Analysis
(0mq(B Io.Writer.print__anon_3010
[42/119] Code Generation
MMMM[270/2998] Linking
(0mq(B debug.Dwarf.LineNumberProgram.addRow
[3811] Semantic Analysis
(0mq(B heap.arena_allocator.ArenaAllocator.allo
[305/447] Code Generation
MMMMM[738/4208] Linking
(0mq(B posix.getenvZ
[5145] Semantic Analysis
(0mq(B array_hash_map.ArrayHashMapUnmanaged(u64
[900/957] Code Generation
MMMMM[4510/4824] Linking
(0mq(B os.windows.TEB
MM]9;4;0/tmp/lean.zig:10:9: error: root source file struct 'cimport' has no member named 'lean_initialize'
    lean.lean_initialize();
    ~~~~^~~~~~~~~~~~~~~~
/home/philip/.cache/zig/o/fb5ee03badfa89d213f1780069b63ab0/cimport.zig:1:1: note: struct declared here
pub const __builtin_bswap16 = @import("std").zig.c_builtins.__builtin_bswap16;
^~~
referenced by:
    callMain [inlined]: /snap/zig/14958/lib/std/start.zig:627:37
    callMainWithArgs [inlined]: /snap/zig/14958/lib/std/start.zig:587:20
    main: /s

D? https://news.ycombinator.com/item?id=43744591
Man I dunno, somehow 

https://github.com/PiIsRational/zat sat solver

https://www.openmymind.net/Zig-Interfaces/

https://github.com/githwxi/ATS-Postiats
go
nim
crystal
ada
common lisp

hare
gleam https://github.com/gleam-lang/gleam erlang variant
odin https://odin-lang.org/

verse
mojo 

ispc https://pharr.org/matt/blog/2018/04/18/ispc-origins

https://vine.dev/docs/  interaction nets

roc https://www.roc-lang.org/
neut https://vekatze.github.io/neut/overview.html

june https://github.com/sophiajt/june/tree/main
MaPLe https://github.com/MPLLang/mpl https://www.cs.cmu.edu/~swestric/24/popl24-par-manage.pdf

hazel https://arxiv.org/abs/2212.06321 Data Layout from a Type-Theoretic Perspective https://hazel.org/
jai
c3 https://github.com/c3lang/c3c

https://nova-lang.net/  more of a rule DSL
https://cuelang.org/ configruation and schema

carbon - next gen c++
koka
https://vale.dev/  probably dead

unison


better bash
https://oils.pub/
https://bpp.sh/

https://www.uiua.org/
https://mlochbaum.github.io/BQN/

https://www.hylo-lang.org/

https://bsky.app/profile/steveklabnik.com/post/3lj3perkaws2x klabnik asking

https://github.com/ChessMax/awesome-programming-languages